In [ ]:
!pip install plotly

Smart City

# Introduction : présentation du projet et des objectifs

Ce projet vise à analyser les données des arbres de la ville de Paris. L'objectif est d'identifier les espèces les plus présentes, leur répartition géographique et d'autres informations pertinentes pour une meilleure gestion du patrimoine arboré

## Résumé des étapes :

1. Clonage du dépôt GitHub Analyse logistique dans Colab.
2. Configuration de l'identité Git avec nom et email.
3. Copie du notebook Gomis_Angele_2_SmartCity_102024.ipynb dans le dépôt.
4. Ajout, commit et envoi du notebook vers GitHub.

# 1. Etape1 : Mise en place de l'environnement

In [ ]:
# Téléchargement des bibliothèques
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


# 2. Chargement des données & premières analyses

In [ ]:
from google.colab import files
uploaded = files.upload()


In [6]:
new_df = pd.read_csv('p2-arbres-fr.csv', sep=';')

FileNotFoundError: [Errno 2] No such file or directory: 'p2-arbres-fr.csv'

In [ ]:
df=new_df.copy()

In [ ]:
# Affichage des colonnes du DataFrame
df.columns

In [ ]:
df.info()

# Traitement des valeurs nulles et des valeurs aberrantes

In [ ]:
# Vérifier le nombre de valeurs NaN par colonne
df.isnull().sum()
# Calculer le pourcentage de valeurs NaN par colonne
nan_percentage = (df.isnull().sum() / len(df) * 100).round(2)

# Trier les pourcentages du plus grand au plus petit
sorted_nan_percentage = nan_percentage.sort_values(ascending=False)

# Afficher les résultats
print("Pourcentages de valeur NaN par colonne")
print(sorted_nan_percentage)

In [ ]:


# Calculate the number of null values per column
nan_counts = df.isnull().sum()

# Calculate the percentage of null values per column
nan_percentage = (nan_counts / len(df) * 100).round(2)

# Create a DataFrame to present the results
nan_df = pd.DataFrame({
    'Nombre de valeurs nulles': nan_counts,
    'Pourcentage de valeurs nulles (%)': nan_percentage
})

# Filter to keep only columns with at least one null value
nan_df = nan_df[nan_df['Nombre de valeurs nulles'] > 0]

# Sort the DataFrame in descending order of the number of null values
nan_df = nan_df.sort_values(by='Nombre de valeurs nulles', ascending=False)

# Now you can check if nan_df is empty
if nan_df.empty:
    print("\nAucune colonne ne contient de valeurs nulles.")
else:
    # Get the names of columns without null values
    no_nan_cols = df.columns[df.isnull().sum() == 0]
    if len(no_nan_cols) > 0:
        print("\nLes colonnes suivantes n'ont pas de valeurs nulles :")
        for col in no_nan_cols:
            print(f"- {col}")
    else:
        print("\nToutes les colonnes contiennent des valeurs nulles.")

# Display the DataFrame
nan_df

In [ ]:
# Colonnes relatives à la circonférence et à la hauteur
target_cols = ['circonference_cm', 'hauteur_m']  # Assurez-vous que les noms des colonnes correspondent exactement

# Parcourir chaque colonne cible
for col in target_cols:
    # Calcul des statistiques de base
    std_dev = df[col].std()
    min_val = df[col].min()
    max_val = df[col].max()
    zero_count = (df[col] == 0).sum()  # Nombre de valeurs égales à 0
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identifier les valeurs aberrantes
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_count = len(outliers)
    outlier_percentage = (outlier_count / len(df)) * 100

    # Afficher les statistiques
    print(f"\nStatistiques pour la colonne '{col}' :")
    print(f"- Écart-type : {std_dev:.2f}")
    print(f"- Valeur minimale : {min_val}")
    print(f"- Valeur maximale : {max_val}")
    print(f"- Nombre de valeurs égales à zéro : {zero_count}")
    print(f"- Nombre de valeurs aberrantes : {outlier_count}")
    print(f"- Pourcentage de valeurs aberrantes : {outlier_percentage:.2f}%")

    # Analyse légère des résultats
    if outlier_percentage > 5:
        print(f"  -> La colonne '{col}' a un pourcentage élevé de valeurs aberrantes ({outlier_percentage:.2f}%). "
              f"Cela peut indiquer une forte variabilité dans les données ou la présence de valeurs extrêmes.")
    else:
        print(f"  -> La colonne '{col}' a un pourcentage faible de valeurs aberrantes ({outlier_percentage:.2f}%). "
              f"Cela suggère une distribution plus homogène sans valeurs extrêmes majeures.")

    if std_dev > (max_val - min_val) / 4:
        print(f"  -> L'écart-type élevé par rapport à l'amplitude (max-min) pour '{col}' indique une dispersion importante.")
    else:
        print(f"  -> L'écart-type relativement faible pour '{col}' montre une faible dispersion autour de la moyenne.")


On remarque ici beaucoup d'arbres dont la valeur minimal est à 0 sur ces 2 colonne cela pourrait etre un code interne, je ne modifie pas cette données, je vérifie/
-- est ce que ce à est sur les meme arbres a chaque fois ou est il dispersé dans le df

## Analyse sur ces 0

In [ ]:
import matplotlib.pyplot as plt

# Données des comptes obtenus
nb_circonference_zero = 25867
nb_hauteur_zero = 39219
nb_both_zero = 25501

# Calcul des seuls circonférence ou hauteur à zéro
only_circonference_zero = nb_circonference_zero - nb_both_zero
only_hauteur_zero = nb_hauteur_zero - nb_both_zero

# Création du diagramme de Venn
fig, ax = plt.subplots(figsize=(8, 6))

# Cercle pour les arbres ayant une circonférence de 0 (en bleu pastel)
circle1 = plt.Circle((0.3, 0.5), 0.3, color="lightblue", alpha=0.5, label="Circonférence = 0")
ax.add_patch(circle1)
ax.text(0.2, 0.5, f"{only_circonference_zero}", ha="center", va="center", fontsize=12, color="black")

# Cercle pour les arbres ayant une hauteur de 0 (en rouge pastel)
circle2 = plt.Circle((0.6, 0.5), 0.3, color="lightcoral", alpha=0.5, label="Hauteur = 0")
ax.add_patch(circle2)
ax.text(0.7, 0.5, f"{only_hauteur_zero}", ha="center", va="center", fontsize=12, color="black")

# Zone de recouvrement pour les deux zéros
ax.text(0.45, 0.5, f"{nb_both_zero}", ha="center", va="center", fontsize=12, color="black")

# Titre et légende
ax.set_title("Distribution des valeurs nulles pour la circonférence et la hauteur des arbres", fontsize=14)
plt.legend(loc="upper left", fontsize=10)
ax.set_aspect("equal")
ax.axis("off")

# Affichage du graphique
plt.show()


Ces arbres avec 0 ... Mais qui etes vous?

In [ ]:
# Séparer les arbres avec et sans valeurs nulles dans les deux colonnes
df_both_zero = df[(df['circonference_cm'] == 0) & (df['hauteur_m'] == 0)]
df_non_zero = df[(df['circonference_cm'] != 0) | (df['hauteur_m'] != 0)]

# Calculer les statistiques descriptives
print("Statistiques descriptives pour les arbres avec circonférence et hauteur = 0")
print(df_both_zero.describe(include='all'))
print("\nStatistiques descriptives pour les arbres avec des valeurs non nulles")
print(df_non_zero.describe(include='all'))

# Comparaison par genre et espèce
genre_comparison = pd.DataFrame({
    'Avec zeros (genre)': df_both_zero['genre'].value_counts(normalize=True),
    'Sans zeros (genre)': df_non_zero['genre'].value_counts(normalize=True)
}).fillna(0)

espece_comparison = pd.DataFrame({
    'Avec zeros (espece)': df_both_zero['espece'].value_counts(normalize=True),
    'Sans zeros (espece)': df_non_zero['espece'].value_counts(normalize=True)
}).fillna(0)

# Afficher les comparaisons de genre et d'espèce
print("\nComparaison de la répartition des genres entre les groupes :")
print(genre_comparison)

print("\nComparaison de la répartition des espèces entre les groupes :")
print(espece_comparison)

# Comparaison par arrondissement
arrondissement_comparison = pd.DataFrame({
    'Avec zeros': df_both_zero['arrondissement'].value_counts(normalize=True),
    'Sans zeros': df_non_zero['arrondissement'].value_counts(normalize=True)
}).fillna(0)

print("\nComparaison de la répartition par arrondissement entre les groupes :")
print(arrondissement_comparison)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Séparer les arbres avec et sans valeurs nulles dans les deux colonnes
df_both_zero = df[(df['circonference_cm'] == 0) & (df['hauteur_m'] == 0)]
df_non_zero = df[(df['circonference_cm'] != 0) | (df['hauteur_m'] != 0)]

# Définir des couleurs pour les graphiques
colors = ["#FFA07A", "#20B2AA"]

# 1. Comparaison des genres
genre_comparison = pd.DataFrame({
    'Avec zeros': df_both_zero['genre'].value_counts(normalize=True),
    'Sans zeros': df_non_zero['genre'].value_counts(normalize=True)
}).fillna(0)

# Diagramme en barres empilé pour la répartition des genres
genre_comparison.plot(kind='bar', stacked=True, color=colors, figsize=(10, 6))
plt.title("Répartition des genres d'arbres avec et sans valeurs nulles")
plt.ylabel("Proportion")
plt.xlabel("Genre")
plt.legend(["Avec zeros", "Sans zeros"])
plt.xticks(rotation=45)
plt.show()

# 2. Comparaison des espèces
espece_comparison = pd.DataFrame({
    'Avec zeros': df_both_zero['espece'].value_counts(normalize=True).head(10),
    'Sans zeros': df_non_zero['espece'].value_counts(normalize=True).head(10)
}).fillna(0)

# Diagramme en barres empilé pour la répartition des espèces
espece_comparison.plot(kind='bar', stacked=True, color=colors, figsize=(10, 6))
plt.title("Top 10 des espèces d'arbres avec et sans valeurs nulles")
plt.ylabel("Proportion")
plt.xlabel("Espèce")
plt.legend(["Avec zeros", "Sans zeros"])
plt.xticks(rotation=45)
plt.show()

# 3. Répartition géographique par arrondissement
arrondissement_comparison = pd.DataFrame({
    'Avec zeros': df_both_zero['arrondissement'].value_counts(normalize=True),
    'Sans zeros': df_non_zero['arrondissement'].value_counts(normalize=True)
}).fillna(0)

# Diagramme en barres empilé pour la répartition par arrondissement
arrondissement_comparison.plot(kind='bar', stacked=True, color=colors, figsize=(10, 6))
plt.title("Répartition par arrondissement des arbres avec et sans valeurs nulles")
plt.ylabel("Proportion")
plt.xlabel("Arrondissement")
plt.legend(["Avec zeros", "Sans zeros"])
plt.xticks(rotation=45)
plt.show()

# 4. Statistiques descriptives supplémentaires (Boxplots)
numeric_cols = ['circonference_cm', 'hauteur_m']  # Adapter pour inclure d'autres colonnes numériques si disponibles
df_both_zero['type'] = 'Avec zeros'
df_non_zero['type'] = 'Sans zeros'
combined_df = pd.concat([df_both_zero, df_non_zero])


# Définir les couleurs pour les graphiques
colors = ["#FFA07A", "#20B2AA"]

# Ajouter une colonne 'type' pour distinguer les groupes
df_both_zero['type'] = 'Avec zeros'
df_non_zero['type'] = 'Sans zeros'
combined_df = pd.concat([df_both_zero, df_non_zero])

# Colonnes numériques à analyser
numeric_cols = ['circonference_cm', 'hauteur_m']

# Boxplots avec échelle logarithmique
for col in numeric_cols:
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=combined_df, x='type', y=col, palette=colors)
    plt.yscale('log')  # Applique l'échelle logarithmique sur l'axe Y
    plt.title(f"Comparaison des {col} entre arbres avec et sans valeurs nulles (échelle log)")
    plt.ylabel(f"{col.capitalize()} (échelle logarithmique)")
    plt.xlabel("Type d'arbre")
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Fonction pour détecter les outliers selon la méthode de Tukey
def detect_outliers(data, col):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    return outliers

# Identifier les outliers pour chaque groupe
outliers_both_zero_circ = detect_outliers(df_both_zero, 'circonference_cm')
outliers_non_zero_circ = detect_outliers(df_non_zero, 'circonference_cm')

outliers_both_zero_haut = detect_outliers(df_both_zero, 'hauteur_m')
outliers_non_zero_haut = detect_outliers(df_non_zero, 'hauteur_m')

# Visualiser les outliers sur un scatterplot pour la circonférence
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_both_zero, x='type', y='circonference_cm', color=colors[0], label="Avec zeros", alpha=0.6)
sns.scatterplot(data=df_non_zero, x='type', y='circonference_cm', color=colors[1], label="Sans zeros", alpha=0.6)
sns.scatterplot(data=outliers_both_zero_circ, x='type', y='circonference_cm', color='red', marker='o', s=100, label="Outliers (Avec zeros)")
sns.scatterplot(data=outliers_non_zero_circ, x='type', y='circonference_cm', color='blue', marker='o', s=100, label="Outliers (Sans zeros)")


## Distinction des valeurs aberrantes des valeurs atypiques


Pour distinguer les valeurs aberrantes des valeurs atypiques dans les colonnes du dataset, voici les approches que l’on peut suivre. Bien que souvent utilisés de manière interchangeable, ces termes représentent deux concepts distincts en statistique et en analyse de données :

Valeurs aberrantes (outliers) :

Ce sont des données situées loin des autres observations et qui peuvent souvent être attribuées à des erreurs de mesure, des valeurs incorrectes ou des événements rares.
On les identifie en général par des techniques comme l’intervalle interquartile (IQR) ou en appliquant des z-scores pour repérer les données très éloignées de la moyenne.
Valeurs atypiques (anomalies) :

Contrairement aux valeurs aberrantes, les valeurs atypiques ne sont pas forcément des erreurs. Ce sont des observations qui, bien que différentes du reste des données, peuvent représenter des sous-groupes significatifs ou des segments distincts.
Par exemple, une très grande circonférence pourrait être typique pour une espèce d’arbres spécifique, sans pour autant être une erreur.
Étapes pour les distinguer
Pour les colonnes circonference_cm et hauteur_m, nous pouvons :

1. Identifier les valeurs aberrantes avec l’intervalle interquartile (IQR) ou les z-scores
IQR : Calculer les quartiles (Q1 et Q3) et définir les valeurs aberrantes comme les points au-delà de
𝑄
1
−
1.5
×
𝐼
𝑄
𝑅
Q1−1.5×IQR et
𝑄
3
+
1.5
×
𝐼
𝑄
𝑅
Q3+1.5×IQR.
Z-score : Calculer le score z (nombre d'écarts-types par rapport à la moyenne) et marquer les données ayant un z-score supérieur à un certain seuil, souvent fixé à 3.
2. Je vais ici isoler et identifier les valeurs atypiques en appliquant un clustering
Utiliser des algorithmes de regroupement (clustering) Isolation Forest (un modèle de machine learning non supervisé pour identifier les anomalies).
Les valeurs atypiques seront identifiées comme des observations appartenant à des clusters distincts ou isolées par rapport aux autres données.

In [ ]:
from sklearn.ensemble import IsolationForest
# Identification des valeurs aberrantes avec l'IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

outliers_circonference = detect_outliers_iqr(df, "circonference_cm")
outliers_hauteur = detect_outliers_iqr(df, "hauteur_m")

# Identification des valeurs atypiques avec Isolation Forest
isolation_forest = IsolationForest(contamination=0.05, random_state=42)
df['anomalie_score'] = isolation_forest.fit_predict(df[['circonference_cm', 'hauteur_m']])

# Les valeurs avec un score de -1 sont marquées comme atypiques par Isolation Forest
atypiques = df[df['anomalie_score'] == -1]

# Affichage des résultats
print("Valeurs aberrantes (IQR) pour la circonférence:", outliers_circonference)
print("Valeurs aberrantes (IQR) pour la hauteur:", outliers_hauteur)
print("Valeurs atypiques (Isolation Forest):", atypiques)


In [ ]:


plt.figure(figsize=(12, 8))

# Carte thermique pour la densité
sns.kdeplot(data=df, x="circonference_cm", y="hauteur_m", fill=True, cmap="Blues", alpha=0.6)

# Superposition des valeurs atypiques
plt.scatter(atypiques['circonference_cm'], atypiques['hauteur_m'], color='green', label='Atypiques', alpha=0.7)

plt.xlabel("Circonférence (cm)")
plt.ylabel("Hauteur (m)")
plt.title("Densité des arbres avec valeurs atypiques")
plt.legend()
plt.show()


. Calcul de l'âge estimé pour chaque arbre
En utilisant des facteurs de croissance moyens par espèce, nous allons ajouter une colonne age_estime dans notre DataFrame. Pour chaque arbre, nous calculons :

A
ˆ
ge estim
e
ˊ
=
Circonf
e
ˊ
rence (cm)
Facteur de croissance (cm/an)
A
ˆ
 ge estim
e
ˊ
 =
Facteur de croissance (cm/an)
Circonf
e
ˊ
 rence (cm)
​

Les taux de croissance moyens sont approximés comme suit :

Chêne : 1,25 cm/an
Érable : 1,5 cm/an
Pin : 0,5 cm/an
Bouleau : 1 cm/an
2. Comparaison des âges estimés avec les distributions pour identifier les valeurs atypiques et aberrantes
Nous allons calculer les valeurs aberrantes et atypiques en analysant les âges estimés pour vérifier si certaines valeurs de circonférence sont réalistes pour l'âge obtenu par ces facteurs.

In [ ]:

# Facteurs de croissance par espèce
facteurs_croissance = {
    'Chêne': 1.25,
    'Érable': 1.5,
    'Pin': 0.5,
    'Bouleau': 1.0,
    # Ajouter d'autres espèces avec des facteurs spécifiques si connus
}

# Fonction pour calculer l'âge estimé
def calcul_age_estime(row):
    facteur = facteurs_croissance.get(row['genre'], 1)  # Utilisation de 1 cm/an si le genre n'est pas dans le dictionnaire
    return row['circonference_cm'] / facteur if facteur > 0 else np.nan

# Application de la fonction pour obtenir l'âge estimé
df['age_estime'] = df.apply(calcul_age_estime, axis=1)

# Identifier les valeurs aberrantes basées sur l'âge estimé
def detect_outliers_by_age(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

outliers_age = detect_outliers_by_age(df, 'age_estime')

# Visualisation : Diagramme de dispersion avec couleurs pour les valeurs aberrantes d'âge
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.scatter(df['circonference_cm'], df['hauteur_m'], color='blue', label='Normaux', alpha=0.5)
plt.scatter(outliers_age['circonference_cm'], outliers_age['hauteur_m'], color='red', label='Valeurs aberrantes d\'âge', alpha=0.7)

plt.xlabel("Circonférence (cm)")
plt.ylabel("Hauteur (m)")
plt.title("Distribution des arbres avec valeurs aberrantes en fonction de l'âge estimé")
plt.legend()
plt.show()


In [ ]:
# Étape 1 : Importer les librairies nécessaires
import plotly.express as px
import plotly.graph_objects as go

# Étape 2 : Sélectionner les colonnes pour le nuage de points
cols_to_plot = ['circonferenceencm', 'hauteurenm']

# Vérifier et convertir les colonnes en numériques si nécessaire
df[cols_to_plot] = df[cols_to_plot].apply(pd.to_numeric, errors='coerce')

# Supprimer les lignes avec des valeurs manquantes dans les colonnes sélectionnées
df_clean = df.dropna(subset=cols_to_plot + ['espece', 'libellefrancais'])

# Étape 3 : Créer le nuage de points principal
fig = px.scatter(
    df_clean,
    x=cols_to_plot[0],
    y=cols_to_plot[1],
    color="espece",
    hover_data=['libellefrancais'],
    title="Nuage de points avec outliers"
)

# Étape 4 : Identifier les outliers avec la méthode IQR
Q1_hauteur = df_clean[cols_to_plot[1]].quantile(0.25)
Q3_hauteur = df_clean[cols_to_plot[1]].quantile(0.75)
IQR_hauteur = Q3_hauteur - Q1_hauteur
lower_bound_hauteur = Q1_hauteur - 1.5 * IQR_hauteur
upper_bound_hauteur = Q3_hauteur + 1.5 * IQR_hauteur

Q1_circonference = df_clean[cols_to_plot[0]].quantile(0.25)
Q3_circonference = df_clean[cols_to_plot[0]].quantile(0.75)
IQR_circonference = Q3_circonference - Q1_circonference
lower_bound_circonference = Q1_circonference - 1.5 * IQR_circonference
upper_bound_circonference = Q3_circonference + 1.5 * IQR_circonference

# Étape 5 : Combiner les outliers des deux colonnes
outliers = df_clean[
    (df_clean[cols_to_plot[1]] < lower_bound_hauteur) | (df_clean[cols_to_plot[1]] > upper_bound_hauteur) |
    (df_clean[cols_to_plot[0]] < lower_bound_circonference) | (df_clean[cols_to_plot[0]] > upper_bound_circonference)
]

# Étape 6 : Vérifier si des outliers ont été détectés
if outliers.empty:
    print("Aucune valeur aberrante détectée.")
else:
    # Étape 7 : Mettre en évidence les outliers dans le graphique
    fig.add_trace(go.Scatter(
        x=outliers[cols_to_plot[0]],
        y=outliers[cols_to_plot[1]],
        mode='markers',
        marker=dict(size=12, color='red', symbol='x'),
        name='Outliers',
        text=outliers['libellefrancais'],
        hovertemplate='<b>%{text}</b><br>Circonférence: %{x}<br>Hauteur: %{y}<extra></extra>'
    ))

# Étape 8 : Afficher le graphique
fig.show()


In [ ]:
# Afficher le nombre d'outliers par espèce
outliers_by_species = outliers['espece'].value_counts()
print(outliers_by_species)

# Visualiser la distribution avec un diagramme en barres
import plotly.express as px
fig = px.bar(outliers_by_species, x=outliers_by_species.index, y=outliers_by_species.values,
             title="Nombre d'outliers par espèce")
fig.show()

In [ ]:
import plotly.express as px

# Sélectionner les espèces à inclure
selected_species = ['x hispanica', 'hippocastanum', 'orientalis']

# Filtrer le DataFrame
filtered_df = df_clean[df_clean['espece'].isin(selected_species)]

# Créer le nuage de points
fig = px.scatter(filtered_df, x='circonferenceencm', y='hauteurenm', color='espece',
                 size='circonferenceencm', hover_data=['libellefrancais'],
                 title="Nuage de points avec outliers (espèces sélectionnées)")
fig.show()

In [ ]:
import plotly.express as px

# Sélectionner les espèces à inclure
selected_species = ['x hispanica', 'hippocastanum', 'orientalis']

# Filtrer le DataFrame
filtered_df = df_clean[df_clean['espece'].isin(selected_species)]

# Calculer le ratio circonférence/hauteur et ajouter une colonne 'ratio'
filtered_df['ratio'] = filtered_df['circonferenceencm'] / filtered_df['hauteurenm']

# Créer un scatter plot avec coloration par espèce
fig = px.scatter(filtered_df, x='circonferenceencm', y='hauteurenm', color='espece',
                 hover_data=['libellefrancais', 'ratio'],
                 title="Circonférence vs. Hauteur par espèce")
fig.show()

In [ ]:
# Filtrer les arbres avec une circonférence supérieure à 700
large_trees = df_clean[df_clean['circonferenceencm'] > 700]

# Trier les arbres par espèce
large_trees_sorted = large_trees.sort_values(by=['espece'])

# Afficher les arbres filtrés et triés
print(large_trees_sorted[['libellefrancais', 'espece', 'circonferenceencm', 'hauteurenm']])

In [ ]:
import plotly.express as px
import pandas as pd

# Sélectionner les espèces à inclure
selected_species = ['x hispanica', 'hippocastanum', 'orientalis']

# Filtrer le DataFrame
filtered_df = df_clean[df_clean['espece'].isin(selected_species)]

# Calculer le ratio circonférence/hauteur et ajouter une colonne 'ratio'
filtered_df['ratio'] = filtered_df['circonferenceencm'] / filtered_df['hauteurenm']

# Créer un scatter plot pour chaque espèce avec les valeurs aberrantes en couleur
for species in selected_species:
    species_df = filtered_df[filtered_df['espece'] == species]

    # Identifier les outliers pour le ratio avec la méthode IQR
    Q1 = species_df['ratio'].quantile(0.25)
    Q3 = species_df['ratio'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = species_df[(species_df['ratio'] < lower_bound) | (species_df['ratio'] > upper_bound)]

    # Créer le scatter plot avec coloration des outliers
    fig = px.scatter(species_df, x='circonferenceencm', y='hauteurenm',
                     color=species_df.index.isin(outliers.index),  # Colorer les outliers
                     color_discrete_map={False: 'blue', True: 'red'},  # Définir les couleurs
                     title=f"Nuage de points pour {species} (outliers en rouge)",
                     hover_data=['libellefrancais', 'ratio'])
    fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Sélectionner les espèces à inclure
selected_species = ['x hispanica', 'hippocastanum', 'orientalis', 'nigra', 'occidentalis']

# Filtrer le DataFrame
filtered_df = df_clean[df_clean['espece'].isin(selected_species)]

# Calculer le ratio circonférence/hauteur et ajouter une colonne 'ratio'
filtered_df['ratio'] = filtered_df['circonferenceencm'] / filtered_df['hauteurenm']

# Créer un DataFrame pour stocker les outliers
all_outliers = pd.DataFrame()

# Parcourir chaque espèce et identifier les outliers
for species in selected_species:
    species_df = filtered_df[filtered_df['espece'] == species]

    # Identifier les outliers pour le ratio avec la méthode IQR
    Q1 = species_df['ratio'].quantile(0.25)
    Q3 = species_df['ratio'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = species_df[(species_df['ratio'] < lower_bound) | (species_df['ratio'] > upper_bound)]

    # Ajouter les outliers au DataFrame all_outliers
    all_outliers = pd.concat([all_outliers, outliers])

# Supprimer les outliers du DataFrame filtré
filtered_df_no_outliers = filtered_df.drop(all_outliers.index)


In [ ]:
all_outliers.info()

2136 arbre sont considéré comme outliers

In [ ]:
import pandas as pd

# ... (code précédent pour créer filtered_df et filtered_df_no_outliers) ...

# Afficher les statistiques descriptives avant et après suppression des outliers
print("Statistiques descriptives avant suppression des outliers :")
print(filtered_df[['circonferenceencm', 'hauteurenm']].describe().round(2))

print("\nStatistiques descriptives après suppression des outliers :")
print(filtered_df_no_outliers[['circonferenceencm', 'hauteurenm']].describe().round(2))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Sélectionner les espèces à inclure
selected_species = ['x hispanica', 'hippocastanum', 'orientalis']

# Filtrer le DataFrame
filtered_df = df_clean[df_clean['espece'].isin(selected_species)]

# Calculer le ratio circonférence/hauteur et ajouter une colonne 'ratio'
filtered_df['ratio'] = filtered_df['circonferenceencm'] / filtered_df['hauteurenm']

# Créer des box plots pour chaque espèce et chaque variable
for variable in ['circonferenceencm', 'hauteurenm', 'ratio']:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='espece', y=variable, data=filtered_df)
    plt.title(f"Distribution de {variable} par espèce")
    plt.xticks(rotation=45, ha='right')  # Rotation des étiquettes de l'axe x
    plt.show()

Le but etant d'organiser les tournée, jene peux supprimer d'arbre de mon df, je prends la décision de conserver ces données et de les remplacer par

In [ ]:
filtered_df_transformed = filtered_df_no_outliers.copy()

In [ ]:
from sklearn.impute import SimpleImputer

# Créer un imputer pour remplacer les valeurs aberrantes par la médiane
imputer = SimpleImputer(strategy='median')

# Ajuster l'imputer aux données et transformer les données
filtered_df_transformed [['circonferenceencm', 'hauteurenm']] = imputer.fit_transform(filtered_df_no_outliers[['circonferenceencm', 'hauteurenm']])

Mes df

In [ ]:

# Calculer les statistiques descriptives pour les trois DataFrames
original_stats = filtered_df[['circonferenceencm', 'hauteurenm']].describe()
no_outliers_stats = filtered_df_no_outliers[['circonferenceencm', 'hauteurenm', 'ratio']].describe()
transformed_stats = filtered_df_transformed[['circonferenceencm', 'hauteurenm', 'ratio']].describe()

# Renommer les index pour les différencier dans le tableau
original_stats.index = pd.MultiIndex.from_product([['Original'], original_stats.index])
no_outliers_stats.index = pd.MultiIndex.from_product([['Sans Outliers'], no_outliers_stats.index])
transformed_stats.index = pd.MultiIndex.from_product([['Transformé'], transformed_stats.index])
transformed_stats


Je valide la transformation

In [ ]:
transformed_stats

In [ ]:
# Remplacer les valeurs aberrantes par la moyenne
filtered_df_transformed['circonferenceencm'] = filtered_df_transformed['circonferenceencm'].fillna(filtered_df_no_outliers['circonferenceencm'].mean())
filtered_df_transformed['hauteurenm'] = filtered_df_transformed['hauteurenm'].fillna(filtered_df_no_outliers['hauteurenm'].mean())

In [ ]:
 # Nombre d'arbres par espèces dont les données ont été transformées
 filtered_df_transformed[filtered_df_transformed.index.isin(all_outliers.index)]['espece'].value_counts()

In [ ]:

print(filtered_df_transformed.columns)

In [ ]:
data = filtered_df_transformed.drop(columns=['complementdresse','numero',"varieteoucultivar"])

In [ ]:
data.sample(5)

In [ ]:
df.describe().round(2)

In [ ]:

# Liste des variables à analyser
variables = ['circonference_cm', 'hauteur_m']
fig, axes = plt.subplots(nrows=1, ncols=len(variables), figsize=(15, 8))
fig.suptitle("Boîtes à Moustaches des Dimensions des Arbres (avec valeurs aberrantes)")

for i, var in enumerate(variables):
    # Création de la boîte à moustaches avec valeurs aberrantes
    axes[i].boxplot(df[var].dropna(), flierprops=dict(marker='o', color='red', markersize=5))
    axes[i].set_title(f"{var.capitalize()} (avec valeurs aberrantes)")
    axes[i].set_ylabel(var.capitalize())
    axes[i].set_xlabel("Arbres")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


## Analyse circonférences & hauteurs
Ces caracteristiques numériques sont capitale dans le processus de nettoyage des données

Pour mieux visualiser les données tout en gardant les valeurs aberrantes, nous pouvons utiliser une échelle logarithmique. Cela permettra de réduire l’impact des valeurs extrêmes et de rendre les données dans les boîtes à moustaches plus visibles.

In [ ]:
# Liste des variables à analyser
variables = ['circonference_cm', 'hauteur_m']
fig, axes = plt.subplots(nrows=1, ncols=len(variables), figsize=(15, 8))
fig.suptitle("Boîtes à Moustaches des Dimensions des Arbres (Échelle Logarithmique)")

for i, var in enumerate(variables):
    # Application d'une transformation logarithmique pour mieux visualiser les données
    # Remplacement des valeurs nulles et des valeurs <= 0 pour éviter les erreurs de log
    data_log = df[var].apply(lambda x: np.log10(x) if x > 0 else np.nan).dropna()

    # Création de la boîte à moustaches avec l'échelle logarithmique
    axes[i].boxplot(data_log, flierprops=dict(marker='o', color='red', markersize=5))
    axes[i].set_title(f"{var.capitalize()} (Échelle Log)")
    axes[i].set_ylabel(f"Log10({var.capitalize()})")
    axes[i].set_xlabel("Arbres")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


Comment dissocier les données atypiques des données aberrantes?

In [ ]:
df.sample(3)

#Analyse complémentaire & Nettoyage
##Boxplots par genre pour chaque arrondissement aberrant

Boxplots par genre dans ces arrondissements spécifiques afin d'explorer la répartition des divers types d’arbres et identifier des particularités dans la distribution pour chaque genre. Cela nous permettra de mieux comprendre les écarts dans ces zones.

In [ ]:
# Création de boxplots individuels pour chaque arrondissement aberrant pour éviter les conflits dus aux genres manquants

for arrdt in arrondissements_aberrants:
    # Filtrer les données pour l'arrondissement en cours
    data_arrdt = df_aberrants[df_aberrants["arrondissement"] == arrdt]

    # Création du boxplot pour la diversité des genres dans cet arrondissement
    plt.figure(figsize=(10, 6))
    data_arrdt.boxplot(column="circonference_cm", by="genre")
    plt.title(f"Distribution de la Circonférence par Genre dans l'Arrondissement: {arrdt}")
    plt.suptitle("")  # Supprime le titre automatique de pandas
    plt.xlabel("Genre")
    plt.ylabel("Circonférence (cm)")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


In [ ]:
# Recréons les boxplots pour chaque arrondissement, en gérant les genres manquants de manière individuelle

# Réinitialisation de la figure pour éviter d'éventuels conflits
fig, axes = plt.subplots(nrows=len(arrondissements_aberrants), ncols=1, figsize=(14, 16), sharex=True)

for i, arrdt in enumerate(arrondissements_aberrants):
    # Filtrer les données pour l'arrondissement en cours
    data_arrdt = df_aberrants[df_aberrants["arrondissement"] == arrdt]

    # Création du boxplot pour la diversité des genres dans cet arrondissement
    # Utilisation de get_group pour s'assurer que chaque genre est bien représenté indépendamment
    data_arrdt.boxplot(column="circonference_cm", by="genre", ax=axes[i])
    axes[i].set_title(f"Arrondissement: {arrdt}")
    axes[i].set_xlabel("Genre")
    axes[i].set_ylabel("Circonférence (cm)")

plt.suptitle("Distribution de la Circonférence par Genre dans les Arrondissements Aberrants", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
df.columns

In [ ]:
import pandas as pd
import plotly.express as px

# Choisir une palette de couleurs plus étendue ou personnalisée
color_discrete_map = {
    "Aesculus": "rgb(31, 119, 180)",
    "Quercus": "rgb(255, 127, 14)",
    "Platanus": "rgb(44, 160, 44)",
    # Ajoutez d'autres genres avec des couleurs spécifiques ici...
}

# Créer le graphique avec une palette de couleurs personnalisée
fig = px.box(
    df,
    x='arrondissement',
    y='hauteur_m',
    color='genre',
    title="Distribution de la Hauteur des Arbres par Genre et Arrondissement",
    labels={'circonference_cm': "Circonférence en cm", 'arrondissement': "Arrondissement"},
    points=False,  # Supprime les points pour ne garder que les boîtes
    color_discrete_map=color_discrete_map  # Applique la palette de couleurs personnalisée
)

# Mettre la légende en bas et ajuster la taille
fig.update_layout(
    legend=dict(
        title="Genres",
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    xaxis_tickangle=45
)

# Afficher le graphique
fig.show()


Maintenant je sais ou chercher les reelle valeurs aberrantes

## Comptages détaillés pour les variétés d'arbres

# 3. Analyse univariée

Définition des variables vu sur opendata.paris.fr/

Les variables numériques

In [ ]:
df.columns

Interpretation des variables

## Variables numériques :

IDBASE : Chaque enregistrement est unique.
NUMERO : Majoritairement '0', ce qui pourrait indiquer une absence de numéro de rue.
IDSTATION : Beaucoup de zéros, possiblement des valeurs manquantes ou non attribuées.


## Variables catégoriques :

TYPEEMPLACEMENT et LIBELLEFRANCAIS : Uniquement 'Arbre', donc peu informatif pour l'analyse.
DOMANIALITE : Trois catégories principales, avec une majorité sur 'Terrains de l'État'.
ARRONDISSEMENT : Répartition des arbres par arrondissement, le 15e étant le plus représenté.
GENRE et ESPECE : Grande diversité, avec certains genres et espèces plus fréquents.

Résumé:
      

```
      # Principales colonnes pertinentes pour l'analyse :

id : Identifiant unique de chaque arbre.
type_emplacement : Type d'emplacement (par exemple, "Arbre").

domanialite : Propriété de l'arbre (ex. "Jardin").
arrondissement : Localisation géographique (ex. "PARIS 7E ARRDT").

libelle_francais : Nom français de l'arbre.
genre et espece : Genre et espèce botanique de l'arbre.

circonference_cm et hauteur_m : Mesures de l'arbre en centimètres et mètres.
stade_developpement : Niveau de développement de l'arbre.

remarquable : Indicateur si l'arbre est remarquable ou non.

geo_point_2d_a et geo_point_2d_b : Coordonnées géographiques.



Colonnes principales:
- id                  : Identifiant unique de l'arbre
- type_emplacement    : Type d'emplacement de l'arbre
- domanialite         : Type de propriété de l'arbre
- arrondissement      : Arrondissement de l'arbre
- complement_adresse  : Adresse complémentaire de l'arbre
- numero              : Numéro d'adresse (inexistant ici)
- lieu                : Lieu exact où l'arbre est situé
- id_emplacement      : Identifiant de l'emplacement
- libelle_francais    : Nom français commun de l'arbre
- genre               : Genre botanique de l'arbre
- espece              : Espèce botanique de l'arbre
- variete             : Variété de l'arbre (quand disponible)
- circonference_cm    : Circonférence de l'arbre en cm
- hauteur_m           : Hauteur de l'arbre en mètres
- stade_developpement : Stade de croissance (ex: jeune, mature)
- remarquable         : Marquage si l'arbre est "remarquable" (valeur booléenne)
- geo_point_2d_a      : Latitude
- geo_point_2d_b      : Longitude

In [ ]:
print('Hiérarchie pour les Arbres :')
print('Arbre -> Genre -> Espèce -> Variété')
df.sample(5)

In [ ]:
# Calcul du nombre unique par niveau de la hiérarchie par arrondissement et par genre
unique_counts_genre = df.groupby(["arrondissement", "genre"]).agg({
    "libelle_francais": "nunique",
    "espece": "nunique",
    "variete": "nunique"
}).rename(columns={
    "libelle_francais": "nb_libelle_francais",
    "espece": "nb_espece",
    "variete": "nb_variete"
}).reset_index()

# Transformation pour obtenir un DataFrame adapté aux barres empilées
unique_counts_genre_pivot = unique_counts_genre.pivot(index="arrondissement", columns="genre", values="nb_libelle_francais").fillna(0)

# Création du graphique en barres empilées pour visualiser la diversité des arbres par genre et par arrondissement
fig, ax = plt.subplots(figsize=(14, 8))

# Création de barres empilées pour chaque genre
unique_counts_genre_pivot.plot(kind='bar', stacked=True, ax=ax, colormap='tab20')

# Personnalisation des éléments du graphique
ax.set_title("Diversité des Arbres par Genre et par Arrondissement", fontsize=16)
ax.set_xlabel("Arrondissement", fontsize=12)
ax.set_ylabel("Nombre Unique de Libellés Français", fontsize=12)
ax.legend(title="Genres", bbox_to_anchor=(1.05, 1), loc='')

# Rotation des noms des arrondissements pour améliorer la lisibilité
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Affichage du graphique
plt.show()


In [ ]:
df.columns

In [ ]:
import pandas as pd
import plotly.express as px

# Choisir une palette de couleurs plus étendue ou personnalisée
color_discrete_map = {
    "Aesculus": "rgb(31, 119, 180)",
    "Quercus": "rgb(255, 127, 14)",
    "Platanus": "rgb(44, 160, 44)",
    # Ajoutez d'autres genres avec des couleurs spécifiques ici...
}

# Créer le graphique avec une palette de couleurs personnalisée
fig = px.box(
    df,
    x='arrondissement',
    y='circonference_cm',
    color='genre',
    title="Distribution de la Circonférence des Arbres par Genre et Arrondissement",
    labels={'circonference_cm': "Circonférence en cm", 'arrondissement': "Arrondissement"},
    points=False,  # Supprime les points pour ne garder que les boîtes
    color_discrete_map=color_discrete_map  # Applique la palette de couleurs personnalisée
)

# Mettre la légende en bas et ajuster la taille
fig.update_layout(
    legend=dict(
        title="Genres",
        orientation="v",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.2
    ),
    xaxis_tickangle=45
)

# Afficher le graphique
fig.show()


In [ ]:
# Choisir une palette de couleurs plus étendue ou personnalisée
color_discrete_map = {
    "Aesculus": "rgb(31, 119, 180)",
    "Quercus": "rgb(255, 127, 14)",
    "Platanus": "rgb(44, 160, 44)",
    # Ajoutez d'autres genres avec des couleurs spécifiques ici...
}

# Créer le graphique avec une palette de couleurs personnalisée
fig = px.box(
    df,
    x='arrondissement',
    y='hauteur_m',
    color='genre',
    title="Distribution de la Hauteur des Arbres par Genre et Arrondissement",
    labels={'circonference_cm': "Circonférence en cm", 'arrondissement': "Arrondissement"},
    points=False,  # Supprime les points pour ne garder que les boîtes
    color_discrete_map=color_discrete_map  # Applique la palette de couleurs personnalisée
)

# Mettre la légende en bas et ajuster la taille
fig.update_layout(
    legend=dict(
        title="Genres",
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    xaxis_tickangle=45
)

# Afficher le graphique
fig.show()


Les arrondissements identifiés avec des valeurs aberrantes de diversité d'arbres sont :

BOIS DE BOULOGNE
BOIS DE VINCENNES
HAUTS-DE-SEINE
PARIS 16E ARRDT
Justification de l'isolement de ces arrondissements
Ces arrondissements présentent des niveaux de diversité d’arbres anormalement élevés ou faibles par rapport aux autres. Leurs valeurs de diversité sont situées dans les 5 % les plus élevés ou les plus faibles, ce qui justifie une analyse plus approfondie.

Analyse complémentaire : Boxplots par genre pour chaque arrondissement aberrant
Je vais créer des boxplots pour chaque genre dans ces arrondissements spécifiques afin d'explorer la répartition des divers types d’arbres et identifier des particularités dans la distribution pour chaque genre. Cela nous permettra de mieux comprendre les écarts dans ces zones. Je vais maintenant générer ces boxplots. ​​

Une erreur est survenue lors de la création des boxplots, vraisemblablement en raison d'un problème d'alignement entre les genres et les arrondissements. Cela peut se produire si certains genres ne sont pas représentés de manière uniforme dans tous les arrondissements sélectionnés, ce qui entraîne un déséquilibre.

Je vais ajuster le code pour m'assurer que chaque boxplot est correctement généré, indépendamment des différences de genres entre les arrondissements. ​​

L'erreur persiste probablement en raison de déséquilibres dans le nombre de genres dans chaque arrondissement, ce qui complique l'affichage simultané. Pour résoudre cela, je vais créer des boxplots pour chaque arrondissement et chaque genre de manière indépendante, sans utiliser d'axes partagés, afin d'éviter ce conflit.

Je vais ajuster l’approche pour afficher les boxplots un par un pour chaque arrondissement aberran

Ces résultats montrent quelques particularités, notamment des valeurs extrêmes surprenantes dans les colonnes `circonference_cm` et `hauteur_m`, ainsi qu'une absence totale de valeurs pour `numero` (d’où les `NaN`).

1. **Colonne `circonference_cm` et `hauteur_m`** :
   - Les valeurs maximales (250255 pour la circonférence et 881818 pour la hauteur) sont très élevées et probablement des erreurs de saisie.
   - Il serait judicieux de filtrer les valeurs aberrantes en fixant une limite raisonnable pour chaque colonne. Par exemple, une circonférence de plus de 1000 cm et une hauteur de plus de 100 m semblent improbables pour des arbres de rue.

2. **Colonne `remarquable`** :
   - Cette colonne contient des valeurs binaires (0 ou 1), où 1 indique probablement un arbre remarquable.
   - Il semble y avoir des valeurs manquantes (`count` de 137039 pour 200137 lignes). Cette variable est capitale pour appoeter un soin particulier aux arbres catégorisé remarquables
3. **Colonnes géographiques `geo_point_2d_a` et `geo_point_2d_b`** :
   - Une visualisation de ces points sur une carte aidera à confirmer la cohérence géographique.

4. **Colonne `numero`** :
   - Elle semble ne contenir aucune donnée.



Les variables catégoriques


In [ ]:
# Sélectionner les variables catégoriques
categorical_cols = df.select_dtypes(include=['object']).columns
print(categorical_cols)

In [ ]:
# Parcourir chaque colonne et afficher le nombre de valeurs uniques et les 10 valeurs les plus fréquentes
for col in df.columns:
    unique_values = df[col].nunique()
    print(f"Le nombre de valeurs uniques pour la variable '{col}' est: {unique_values}")
    print(f"Les 10 valeurs les plus fréquentes pour '{col}' sont:\n{df[col].value_counts(dropna=False).head(10)}\n")


In [ ]:
# Liste des variables catégoriques
categorical_cols = df.select_dtypes(include=['object']).columns
print(f"Les variables catégoriques sont :\n{categorical_cols}\n")

# Dictionnaire pour stocker les DataFrames
dict_value_counts = {}

for col in categorical_cols:
    value_counts = df[col].value_counts(dropna=False).reset_index()
    value_counts.columns = [col, 'Counts']
    dict_value_counts[col] = value_counts

# Exemple d'affichage pour la variable 'genre'
df_genre_counts = dict_value_counts['genre']
df_genre_counts.head(15)


Remarques :

Variable 'TYPE' et 'SOUS_CATEGORIE' : Ces variables semblent vides ou non renseignées, ce qui explique le nombre de valeurs uniques à 0 et l'absence de valeurs dans 'Top_Valeurs'.
Variable 'VARIETE' : La valeur 'NaN' indique des valeurs manquantes. Le champ vide '' représente des chaînes de caractères vides.

In [ ]:

# Liste pour stocker les résultats
summary_list = []

# Parcourir chaque colonne
for col in df.columns:
    unique_values = df[col].nunique()
    top_values = df[col].value_counts(dropna=False).head(10)
    # Convertir les top valeurs en chaîne de caractères
    top_values_str = ', '.join([f"{idx}: {count}" for idx, count in top_values.items()])
    summary_list.append({
        'Variable': col,
        'Nb_Valeurs_Uniques': unique_values,
        'Top_Valeurs': top_values_str
    })

# Créer le DataFrame résumé
df_summary = pd.DataFrame(summary_list)

# Afficher le DataFrame
df_summary


In [ ]:
df.sample(5)

In [ ]:
# Sélectionner les variables numériques
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
print(numerical_cols)

In [ ]:
# Statistiques descriptives des variables numériques
df[numerical_cols].describe().round(2)

In [ ]:
sns.pairplot(df[numerical_cols])
plt.show(

In [ ]:


# Calculate the number of null values per column
nan_counts = df.isnull().sum()

# Calculate the percentage of null values per column
nan_percentage = (nan_counts / len(df) * 100).round(2)

# Create a DataFrame to present the results
nan_df = pd.DataFrame({
    'Nombre de valeurs nulles': nan_counts,
    'Pourcentage de valeurs nulles (%)': nan_percentage
})

# Filter to keep only columns with at least one null value
nan_df = nan_df[nan_df['Nombre de valeurs nulles'] > 0]

# Sort the DataFrame in descending order of the number of null values
nan_df = nan_df.sort_values(by='Nombre de valeurs nulles', ascending=False)

# Now you can check if nan_df is empty
if nan_df.empty:
    print("\nAucune colonne ne contient de valeurs nulles.")
else:
    # Get the names of columns without null values
    no_nan_cols = df.columns[df.isnull().sum() == 0]
    if len(no_nan_cols) > 0:
        print("\nLes colonnes suivantes n'ont pas de valeurs nulles :")
        for col in no_nan_cols:
            print(f"- {col}")
    else:
        print("\nToutes les colonnes contiennent des valeurs nulles.")

# Display the DataFrame
nan_df

Les colonnes "numero",variété et complement d'adresse me semble inutilisable

# Interprétation des variables :

# df's

In [ ]:
#df de base = df
#df des valeurs nulles = nan_df
#df valeur aberrantes = all_outliers
#df transformation = filtered_df_transformed
#df final = data

In [ ]:
data

# Visualisation géospatiale : Carte interactive des arbres

In [ ]:
!pip install plotly pandas
import plotly.express as px
import pandas as pd

# Convertir la colonne 'geo_point_2d' en deux colonnes 'latitude' et 'longitude'
df_clean['latitude'] = df_clean['geo_point_2d'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df_clean['longitude'] = df_clean['geo_point_2d'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 0 else None)

# Supprimer les lignes avec des valeurs nulles pour la latitude et la longitude
df_clean = df_clean.dropna(subset=['latitude', 'longitude'])

# Sélectionner les espèces à inclure
selected_species = ['x hispanica', 'hippocastanum', 'orientalis']

# Filtrer le DataFrame
filtered_df = df_clean[df_clean['espece'].isin(selected_species)]

# Grouper par espèce et coordonnées pour compter le nombre d'arbres
grouped_df = filtered_df.groupby(['espece', 'latitude', 'longitude'])['idbase'].count().reset_index(name='nombre_arbres')

# Créer la carte interactive
fig = px.scatter_mapbox(grouped_df,
                        lat='latitude',
                        lon='longitude',
                        color='espece',  # Colorer les points par espèce
                        size='nombre_arbres',  # Taille des points en fonction du nombre d'arbres
                        hover_name='espece',  # Afficher l'espèce au survol
                        hover_data={'latitude': False, 'longitude': False, 'nombre_arbres': True}, # Afficher le nombre d'arbres dans l'infobulle
                        zoom=10,  # Niveau de zoom initial
                        mapbox_style="open-street-map")  # Style de la carte

fig.show()

# Analyse de la biodiversité : Calculs et visualisation de la diversité des espèces

Mes observations

# Organisation des tournées

Quels sont les facteurs les plus importants à prendre en compte pour organiser les tournées (distance, espèce, circonférence, hauteur, etc.) ?
Comment regrouper les arbres pour former des tournées (par espèce, par zone géographique, par taille, etc.) ?
Y a-t-il des contraintes spécifiques à respecter (nombre maximum d'arbres par tournée, distance maximale entre les arbres, etc.) ?



Recherche de données concernant les espèces sur le web (Besoin en eau)

# Conclusion et synthèse : Résumé des résultats et implications pour l'optimisation des tournées